### Data Import

In [4]:
import pandas as pd

nba_data = pd.read_csv(r"seasons_clean.csv")
nba_data.rename(columns={"WS/48":"WS48"}, inplace=True)
nba_data.head(2)

,Season,Player,Pos,Age,Tm,G,GS,MPG,PPG,RPG,...,TS%,USG%,OWS,DWS,WS,WS48,OBPM,DBPM,BPM,VORP
0,2009,Arron Afflalo,SG,24.0,DEN,82.0,75.0,27.085366,8.829268,3.073171,...,0.576,14.0,2.8,1.4,4.3,0.092,0.8,-0.5,0.3,1.3
1,2009,Alexis Ajinca,C,21.0,CHA,6.0,0.0,5.000000,1.666667,0.666667,...,0.479,19.3,-0.1,0.0,0.0,-0.013,-8.5,-2.0,-10.4,-0.1


### Selecting columns/stats to keep in dataset 

In [5]:
stat_list = [7,0,1,3,5,6,8,9,10,11,12,13,15,17,18,20,21,23,24,26,27,28]
output_list = [0,25,30,31,34,35]

print(f'Stat list \n*******\n{nba_data.columns[stat_list]}')
print(f'\n\nOutput list \n*******\n{nba_data.columns[output_list]}')

Stat list 
*******
Index(['MPG', 'Season', 'Player', 'Age', 'G', 'GS', 'PPG', 'RPG', 'APG', 'SPG',
       'BPG', 'TOPG', 'FG', 'FG%', '3P', '3P%', 'FT', 'FT%', 'PFPG', 'TS%',
       'USG%', 'OWS'],
      dtype='object')


Output list 
*******
Index(['Season', 'PER', 'WS', 'WS48', 'BPM', 'VORP'], dtype='object')


### Defining X y datasets 

In [6]:
# Defining dataset of chosen stats 
# (filtering out players with less than half a quarter of play)
nba_X = nba_data.loc[nba_data['MPG'] > 6].iloc[:,stat_list]
nba_X = nba_X[nba_X['Season'] != 2018].drop(columns=["Season","Player"])

In [7]:
# Reserving the 2018 season for "production"
nba_2018_prod = nba_data.loc[nba_data['MPG'] > 6].iloc[:,stat_list]
nba_2018_prod = nba_2018_prod[nba_2018_prod['Season'] == 2018].drop(columns=["Season"])

In [8]:
# Defining output dataset for y_train/y_test of chosen outputs 
# (filtering out players with less than half a quarter of play)
nba_y = nba_data.loc[nba_data['MPG'] > 6].iloc[:,output_list]
nba_y = nba_y[nba_y['Season'] != 2018].drop(columns=["Season"])

### Create and Save the Model 

In [12]:
import matplotlib.pyplot as plt

from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

from joblib import dump, load

@ignore_warnings(category=ConvergenceWarning)
def Create_Model(nba_X,nba_y):
    
    for j in range(0,len(output_list)-1):
        
        output = nba_y.columns[j]
        output_data = nba_y.iloc[:,j]

        ### Building X and y dataframe 

        # Defining inputs
        X = nba_X

        # Defining outputs
        y = output_data.values.reshape(-1)

        print("*********************************")
#         Create_Model(X,y)

        ### Starting data-modeling process 

        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    #     ### Select Player 

    #     # Selecting a random player from test data set
    #     X_player = X_test.sample(n=1, random_state=rand)

    #     season,player,pos = nba_data.iloc[X_player.index.values[0],0:3]
    #     print(f'The {output} for {player} ({pos}, {season})')

        ### Scaling data 

        # Create a StandardScaler model and fit it to the training data
        X_scaler = StandardScaler().fit(X_train)

        # Transform the training and testing data using the X_scaler
        X_train_scaled = X_scaler.transform(X_train)
        X_test_scaled = X_scaler.transform(X_test)

        ### Model fit and score

        from sklearn.neural_network import MLPRegressor
        model = MLPRegressor()   

        model.fit(X_train_scaled, y_train)
        training_score = model.score(X_train_scaled, y_train)
        testing_score = model.score(X_test_scaled, y_test)

        print(f"{output} Training Score: {training_score.round(3)}")
        print(f"{output} Testing Score: {testing_score.round(3)}")
        print(f'Saved: model_NBA_{output}.joblib')

        ### SAVE MODEL

        dump(model, f'Model/model_NBA_{output}.joblib') 

    #     # Permutation Importance
    #     r = permutation_importance(model, X_test_scaled, y_test,
    #                                n_repeats=30,
    #                                random_state=0)

    #     for i in r.importances_mean.argsort()[::-1]:
    #         if r.importances_mean[i] - 2 * r.importances_std[i] > 0:   
    #             print(f"Importance (Highest): {X_test.columns[i]}({r.importances_mean[i]:.3f})\n"
    #                   f"Importance ({X_test.columns[0]}): ({r.importances_mean[0]:.3f})")

    #         # Break in order to print only the highest importance stat
    #         break 



    #     ### Running the Loop

    #     max_mpg = int(nba_X["MPG"].max())
    #     PER_list = list()

    #     # Building predictions based on each minute of MPG
    #     for i in range(7,max_mpg+1):
    #         X_player["MPG"] = i
    #         X_player_scaled = X_scaler.transform(X_player)
    #         y_player = model.predict(X_player_scaled)

    #         PER_list.append(y_player[0])

    #     ### PLOT THE RESULTS

    #     plt.scatter(range(7,max_mpg+1), PER_list, c="blue", label=f"{output} by Minute")
    #     plt.legend()
    #     plt.title(f"Test Data - MPG vs. {output} for {player}")
    #     plt.savefig(fname=f"Figures/{output}_test",facecolor="white")

    #     ### Clear the figure for loop

    #     plt.clf()
        print("*********************************")


In [25]:
@ignore_warnings(category=ConvergenceWarning)
def Regressor_test(nba_X,nba_y):
    
    for j in range(0,len(output_list)-1):
        
        output = nba_y.columns[j]
        output_data = nba_y.iloc[:,j]

        ### Building X and y dataframe 

        # Defining inputs
        X = nba_X

        # Defining outputs
        y = output_data.values.reshape(-1)

        print("*********************************")
#         Create_Model(X,y)

        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

        ### Scaling data 
        # Create a StandardScaler model and fit it to the training data
        X_scaler = StandardScaler().fit(X_train)

        # Transform the training and testing data using the X_scaler
        X_train_scaled = X_scaler.transform(X_train)
        X_test_scaled = X_scaler.transform(X_test)

        ### Select Player 

        # Selecting a random player from test data set
        X_player = X_test.sample(n=1, random_state=rand)

#         if player == 0:
#             # Selecting a random player from test data set
#             player = X_test.sample(n=1, random_state=rand)
#         else:
#             player = player

        season,player,pos = nba_data.iloc[X_player.index.values[0],0:3]
        



        ### Model fit and score

    #     from sklearn.neural_network import MLPRegressor
    #     model = MLPRegressor()   

    #     model.fit(X_train_scaled, y_train)
    #     training_score = model.score(X_train_scaled, y_train)
    #     testing_score = model.score(X_test_scaled, y_test)

    #     print(f"Training Score: {training_score.round(3)}")
    #     print(f"Testing Score: {testing_score.round(3)}")

    #     ### SAVE MODEL

    #     dump(model, f'Model/model_NBA_{output}.joblib') 

        ### LOAD Model
        model_NBA_load = load(f'Model/model_NBA_{output}.joblib') 
        print(f'Load: model_NBA_{output}.joblib')
        print(f'The {output} for {player} ({pos}, {season})')
        
        # Permutation Importance
        r = permutation_importance(model_NBA_load, X_test_scaled, y_test,
                                   n_repeats=30,
                                   random_state=0)

        for i in r.importances_mean.argsort()[::-1]:
            if r.importances_mean[i] - 2 * r.importances_std[i] > 0:   
                print(f"Importance (Highest): {X_test.columns[i]}({r.importances_mean[i]:.3f})\n"
                      f"Importance ({X_test.columns[0]}): ({r.importances_mean[0]:.3f})")

            # Break in order to print only the highest importance stat
            break 



    #     ### Running the Loop

    #     max_mpg = int(nba_X["MPG"].max())
    #     PER_list = list()

    #     # Building predictions based on each minute of MPG
    #     for i in range(7,max_mpg+1):
    #         X_player["MPG"] = i
    #         X_player_scaled = X_scaler.transform(X_player)
    #         y_player = model_NBA_load.predict(X_player_scaled)

    #         PER_list.append(y_player[0])

    #     ### PLOT THE RESULTS

    #     plt.scatter(range(7,max_mpg+1), PER_list, c="blue", label=f"{output} by Minute")
    #     plt.legend()
    #     plt.title(f"Test Data - MPG vs. {output} for {player}")
    #     plt.savefig(fname=f"Figures/{output}_test",facecolor="white")

    #     ### Clear the figure for loop

    #     plt.clf()
        print("*********************************")


### Evaluate MPG impact on Selected Output Stats, and Plot 

In [15]:
Create_Model(nba_X,nba_y)

# for j in range(0,len(output_list)-1):
        
#     output = nba_y.columns[j]
#     output_data = nba_y.iloc[:,j]

#     ### Building X and y dataframe 

#     # Defining inputs
#     X = nba_X

#     # Defining outputs
#     y = output_data.values.reshape(-1)
    
#     print("*********************************")
#     Create_Model(X,y)
#     print("*********************************")

*********************************
PER Training Score: 0.983
PER Testing Score: 0.979
Saved: model_NBA_PER.joblib
*********************************
*********************************
WS Training Score: 0.982
WS Testing Score: 0.971
Saved: model_NBA_WS.joblib
*********************************
*********************************
WS48 Training Score: 0.599
WS48 Testing Score: 0.498
Saved: model_NBA_WS48.joblib
*********************************
*********************************
BPM Training Score: 0.937
BPM Testing Score: 0.9
Saved: model_NBA_BPM.joblib
*********************************
*********************************
VORP Training Score: 0.974
VORP Testing Score: 0.942
Saved: model_NBA_VORP.joblib
*********************************


In [26]:
Regressor_test(nba_X,nba_y)

# from random import randrange

# # Random number for random player selection in Regressor()
# rand = randrange(1, 1000)

# for j in range(0,len(output_list)-1):
        
#     output = nba_y.columns[j]
#     output_data = nba_y.iloc[:,j]

#     ### Building X and y dataframe 

#     # Defining inputs
#     X = nba_X

#     # Defining outputs
#     y = output_data.values.reshape(-1)
    
#     print("*********************************")
#     Regressor(X,y)
#     print("*********************************")

*********************************
Load: model_NBA_PER.joblib
The PER for Hedo Turkoglu (SF, 2011)
Importance (Highest): USG%(0.673)
Importance (MPG): (0.074)
*********************************
*********************************
Load: model_NBA_WS.joblib
The WS for Hedo Turkoglu (SF, 2011)
Importance (Highest): OWS(0.851)
Importance (MPG): (0.115)
*********************************
*********************************
Load: model_NBA_WS48.joblib
The WS48 for Hedo Turkoglu (SF, 2011)
Importance (Highest): OWS(3.137)
Importance (MPG): (1.366)
*********************************
*********************************
Load: model_NBA_BPM.joblib
The BPM for Hedo Turkoglu (SF, 2011)
Importance (Highest): SPG(0.448)
Importance (MPG): (0.089)
*********************************
*********************************
Load: model_NBA_VORP.joblib
The VORP for Hedo Turkoglu (SF, 2011)
Importance (Highest): PPG(0.867)
Importance (MPG): (0.074)
*********************************


# Mean Efficiency Rating 

### League Mean Data 

In [ ]:
def Importance_MER(nba_2018_prod, nba_y):
    
    mean_data = pd.DataFrame(nba_2018_prod.mean()).T
    mean_output = pd.DataFrame(nba_y.mean()).T
    
    rows = []
    for j in range(0,len(output_list)-1):

        output = nba_y.columns[j]
        output_data = mean_output.iloc[:,j]

        # Defining inputs
        mean_data_test = mean_data

        ### LOAD Model
        model_NBA_load = load(f'Model/model_NBA_{output}.joblib') 

        ### Scaling data 
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
        X_scaler = StandardScaler().fit(X_train)

        # Gathering player data
        mean_data_scaled = X_scaler.transform(mean_data_test)
        y_mean = model_NBA_load.predict(mean_data_scaled)


        print("*****************")
        print(f"{output}")

        y_mean_value = y_mean[0]
        print(f"Predicted = {y_mean_value:.2f}")

        y_actual = mean_output.iloc[0][output]
        print(f"Actual = {y_actual:.2f}")

        dif = ((y_mean_value - y_actual)/y_actual)*100
        print(f"MER: {dif:.2f}%")

        print("*****************")

        rows.append([output,round(y_mean_value,2), round(y_actual,2), round(dif,1)])

    df = pd.DataFrame(rows, columns=['Output','Predicted','Actual','% Diff'])
    df

In [ ]:
Importance_MER(nba_2018_prod, nba_y)

# Production 

### Setting dataset for Production 

In [ ]:
def SelectPlayer(x):
    
    # Enter player as string in function above
    player = x
    player_prod = nba_2018_prod[nba_2018_prod['Player'] == player].drop(columns=["Player"])
    
    Production(player_prod)

### Evaluate MPG impact on Selected Output Stats, and Plot 

In [ ]:
import numpy as np
def Production(player_prod):
    # Setting Players actual MPG value
    v = player_prod['MPG'].values[0]
    print(v)
    total_max = 0
    total_min = 0
    
    for j in range(0,len(output_list)-1):

        output = nba_y.columns[j]
        output_data = nba_y.iloc[:,j]

        ### Building X and y dataframe 

        # Defining inputs
        X_prod_2018 = player_prod

        ### LOAD Model

        model_NBA_load = load(f'Model/model_NBA_{output}.joblib') 

        pos = player_prod.iloc[0,2]
        print(f'Evaluated {output} for {player} (2018)')

        ### Scaling data 

        # Create a StandardScaler model and fit it to the training data
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=rand)
        X_scaler = StandardScaler().fit(X_train)

        ### Running the MPG Loop

        max_mpg = int(nba_X["MPG"].max())
        PER_list = list()

        # Building predictions based on each minute of MPG
        for i in range(7,max_mpg+1):

            # Gathering player data
            player_prod["MPG"] = i
            X_player_scaled_2018 = X_scaler.transform(player_prod)
            y_player_2018 = model_NBA_load.predict(X_player_scaled_2018)

            PER_list.append(y_player_2018[0])

        # Scale results
        myInt = max(PER_list)
        PER_list[:] = [x / myInt for x in PER_list]
        
        # Grabbing max and min values to set for vertical line
        total_max = max(total_max,max(PER_list))
        total_min = min(total_min,min(PER_list))
        
        
        ### PLOT THE RESULTS
        
        colors = ["b","g","r","c","m"]
        
        plt.scatter(range(7,max_mpg+1), PER_list, c=colors[j], label=f"{output}")
        plt.title(f"2018 - MPG Impact for {player}")
    plt.vlines(v, total_min, total_max, label=f'MPG')
    plt.legend(loc="lower right")
    plt.savefig(fname=f"Figures/AllStats_for_{player}",facecolor="lightsteelblue")

    ### Clear the figure for loop

    plt.clf()
    print("********************")

In [ ]:
raptors = ["OG Anunoby","Marc Gasol","Danny Green", "Serge Ibaka", "Kawhi Leonard",
           "Jeremy Lin","Kyle Lowry","Norman Powell", "Pascal Siakam", 
           "Fred VanVleet", "Delon Wright"]

#     ### Select Player 
    
#     # Selecting a random player from test data set
#     X_player = X_test.sample(n=1, random_state=rand)

In [ ]:
# SelectPlayer(raptors[2])
for player in raptors:
    
    SelectPlayer(player)